In [1]:
!python -m pip install --upgrade pip -q -U
!pip install -q -U scipy
# !pip install einops #needed for Phi-2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 771.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 34.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
# Required when training models/data that are gated on HuggingFace, and required for pushing models to HuggingFace
!pip install huggingface_hub -q -U
from huggingface_hub import notebook_login

notebook_login()

In [3]:
cache_dir='.' #means models will be downloaded into the current directory

In [4]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import os
# cache_dir = "/content/drive/My Drive/huggingface_cache"
# os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists

# Installation

In [5]:
!pip install git+https://github.com/huggingface/transformers.git -q -U #Necessary for merging LoRA adapters onto quantized models.
# !pip install -q -U transformers # if you are facing issues with the dev branch above

!pip install accelerate -q -U

# # Install peft to allow for LoRA fine-tuning
# !pip install -q -U peft

# # Install bitsandbytes for quantized fine-tuning
# !pip install -q -U bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.7 MB/s eta 0:00:00


## Downloading a Repo

In [27]:
from huggingface_hub import snapshot_download
import os

#hub_model_path = "Trelis/TinyLlama-1.1B-Chat-v1.0-bf16"
# hub_model_path = "meta-llama/Llama-2-7b-chat-hf"
hub_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

local_model_path = cache_dir + '/' + hub_model_path
#downloading full folder then loading model from local
print(local_model_path)

./TinyLlama/TinyLlama-1.1B-Chat-v1.0


In [7]:
repo_path = snapshot_download(
    repo_id=hub_model_path,
    cache_dir=cache_dir,
    local_dir=local_model_path,
    local_dir_use_symlinks=False)

print(f"Repository downloaded to: {local_model_path}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

eval_results.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Repository downloaded to: ./TinyLlama/TinyLlama-1.1B-Chat-v1.0


# Load the Model

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# # COMMENT this in for quantization in 4bit (nf4)!
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    # quantization_config=bnb_config, # COMMENT this in for quantization in 4bit (nf4)!
    device_map='auto', #loads automatically to gpu if there is one.
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    cache_dir=cache_dir)

tokenizer = AutoTokenizer.from_pretrained(local_model_path,use_fast=True,trust_remote_code=True)

In [29]:
#print(model)
print(tokenizer)

LlamaTokenizerFast(name_or_path='./TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


### Prepare for LoRA fine-tuning

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
## Parameter Efficient Fine Tuning (PEFT), specifically, Low Rank Adaptation (LoRA).

from peft import LoraConfig, get_peft_model

config = LoraConfig( #matching the Llama recipe, but with added modules
    r=8,
    lora_alpha=32,
    target_modules=[
              "self_attn.q_proj",
              "self_attn.k_proj",
              "self_attn.v_proj",
              "self_attn.o_proj",
              "mlp.gate_proj",
              "mlp.up_proj",
              "mlp.down_proj",
              ],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

model_with_lora = get_peft_model(
    model,
    config,
)

print_trainable_parameters(model_with_lora)

trainable params: 6307840 || all params: 621914112 || trainable%: 1.0142622394778524


## Typically here you:
- Set up the tokenizer
- Set up padding
- Set up the dataset
- Run evaluation before training
- Set up training
- Run evaluation after training

# Push Adapters and Model to Hub

In [31]:
org = 'Asher'
new_hub_model_path = org + '/' + hub_model_path.split("/")[-1] + '-push-demo'
print(f"Setting up to push the model to {new_hub_model_path} on HuggingFace Hub")

TinyLlama-1.1B-Chat-v1.0
Setting up to push the model to Asher/TinyLlama-1.1B-Chat-v1.0-push-demo on HuggingFace Hub


## Push Adapters to Hub

In [ ]:
# If you have done LoRA fine-tuning
new_hub_model_adapters_path = new_hub_model_path + "-adapters"
print(f"Setting up to push the adapters to {new_hub_model_adapters_path} on HuggingFace Hub")

Setting up to push the adapters to Trelis/TinyLlama-1.1B-Chat-v1.0-bf16-push-demo-adapters on HuggingFace Hub


In [ ]:
### Typically you do one of three things

# # Option A: Pick an adapter from somewhere during your fine-tuning
# adapter_to_push = save_dir + '/checkpoint-32'

# # Option B: Grab an adapter you trained before from HuggingFace Hub
# adapter_to_push = "Trelis/Llama-2-7b-chat-hf-touch-rugby-rules-adapters" #uncomment if you want to grab an adapter from the hub

# Apply the desired adapter to the base model - Required for Option A or Option B.

# # load peft model with the chosen adapter
# model_to_push = PeftModel.from_pretrained(
#     model,
#     adapter_to_push,
# )

# Option C: the adapter is your model_and_lora (yes, this is confusing...), i.e. you are going to push the model_with_lora as it is at the very last step of training.
adapter_to_push = model_with_lora
model_to_push = model_with_lora

In [ ]:
# Save the adapter model
adapter_to_push.save_pretrained(new_hub_model_adapters_path, token=True)

In [ ]:
# Push the model adapters to the hub
# if running this, the peft base model needs to be re-named to refer to a model on the hub
adapter_to_push.push_to_hub(new_hub_model_adapters_path, token=True, safe_serialization=True)

adapter_model.safetensors:   0%|          | 0.00/25.3M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Trelis/TinyLlama-1.1B-Chat-v1.0-bf16-push-demo-adapters/commit/db376ccf7f7b275c7218b42057ccdb87c6020a38', commit_message='Upload model', commit_description='', oid='db376ccf7f7b275c7218b42057ccdb87c6020a38', pr_url=None, pr_revision=None, pr_num=None)

## Merge Adapters

In [ ]:
# Added Option X: Reload a base model in 16-bit precision and then merge.
# The motivation is if you have trained with quantization but want a full precision model to inference using Text Generation Inference OR you want to quantize to GGUF or AWQ. This will hurt precision a little.
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map='auto', #loads automatically to gpu if there is one. It can be useful to load onto cpu if using a free Colab notebook as that gives more RAM.
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    cache_dir=cache_dir)

from peft import PeftModel

# load peft model with the chosen adapter
model_to_push = PeftModel.from_pretrained(
    model,
    new_hub_model_adapters_path,
)

In [ ]:
model_to_push = model_to_push.merge_and_unload() # merge adapters with the base model. This will hurt precision a little if you are merging a quantized model.

## Push Model to Hub

In [32]:
# # ONLY RUN THIS CELL IF YOU DID *NOT* MERGE ADAPTERS, i.e. you did a fine-tuning without LoRA
model_to_push = model

In [33]:
#Save the model locally (using the same path as will be used on the hub
model_to_push.save_pretrained(new_hub_model_path)

In [38]:
print(type(new_hub_model_path))

<class 'str'>


In [36]:
model_to_push.push_to_hub(new_hub_model_path, token=True, max_shard_size="5GB", safe_serialization=True)

IsADirectoryError: [Errno 21] Is a directory: 'Asher/TinyLlama-1.1B-Chat-v1.0-push-demo'

## Push Tokenizer to Hub

In [37]:
## Re-load a tokenizer (uncommon)
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

# save the tokenizer
tokenizer.save_pretrained(new_hub_model_path)

# push the tokenizer to hub
tokenizer.push_to_hub(new_hub_model_path, token=True)

IsADirectoryError: [Errno 21] Is a directory: 'Asher/TinyLlama-1.1B-Chat-v1.0-push-demo'

## (Alternative) Upload a folder to the Hub

In [ ]:
from huggingface_hub import HfApi, upload_folder, create_branch

# Initialize the HfApi class
api = HfApi()

# # Optionally, create a new branch for 'nf4'. Beware this will copy all files from main.
# create_branch(repo_id=new_hub_model_path, repo_type="model", branch="nf4")

# Upload the entire folder to the specified branch in the repository
upload_folder(
    folder_path=new_hub_model_path,
    repo_id=new_hub_model_path,
    repo_type="model",  # Assuming it's a model; can be "dataset" or "space" as well
    # revision="nf4",  # Specify the branch you want to push to
    token=True,
)

print(f"Uploaded contents of {new_hub_model_path} to {new_hub_model_path} on HuggingFace Hub")

Uploaded contents of Trelis/TinyLlama-1.1B-Chat-v1.0-push-demo to Trelis/TinyLlama-1.1B-Chat-v1.0-push-demo on HuggingFace Hub
